# LangChain Basic
**2024 삼성전자 무선사업부 생성형 AI 교육**

1. 패키지 설치
2. 기본 패키지 & 환경 설정
3. LLM
4. Embedding
5. Prompt Template
6. Fewshot Prompt Template
7. Chain + Memory 

In [ ]:
%pip install tiktoken
%pip install langchain
%pip install -U langchain-openai
%pip install langchain_community
%pip install langchain_core

In [ ]:
# 2. 기본 패키지 설정 & AzureOpenAI 환경설정

# 환경 변수 설정!! 
import os
import langchain

from config_azure import (
    AZURE_OPENAI_API_VERSION,
    AZURE_OPENAI_ENDPOINT,
    AZURE_OPENAI_KEY
)
# lagchain 사용시 반드시 환경변수 값 사용!!
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["AZURE_OPENAI_API_KEY"] = AZURE_OPENAI_KEY
os.environ["AZURE_OPENAI_ENDPOINT"] = AZURE_OPENAI_ENDPOINT
os.environ["OPENAI_API_VERSION"] = AZURE_OPENAI_API_VERSION

In [ ]:
# 3-1. OpenAI LLM(CompletionModel 연결)

from langchain_openai import AzureOpenAI

com_llm = AzureOpenAI(
    azure_deployment="gpt-35-turbo-instruct",
    max_tokens=128,
    temperature=0.9)

In [ ]:
com_llm.invoke("오늘 저녁 메뉴는")

In [ ]:
# 3-2. OpenAI LLM(Chat Model 연결)

from langchain_openai import AzureChatOpenAI

chat_llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    max_tokens=256,
    temperature=0.9,
)

In [ ]:
chat_llm.invoke("오늘 저녁 메뉴는").content

In [ ]:
# 3-3. Chat llm 사용

from langchain.schema import (
    SystemMessage, # system
    AIMessage, # assistant
    HumanMessage # user
)

messages = [
    SystemMessage(content="RPG 게임 캐릭터 재키, 성을 지키는 용감한 용사로 유저의 질문에 반말 구어체로 짧게 답한다"),
    HumanMessage(content="안녕?"),
    AIMessage(content="용감한 자여, 전설의 성에 온걸 환영한다!"),
    HumanMessage(content="너의 이름은 뭐니?"),
    AIMessage(content="사람들은 나를 재키라 부르지. 성을 지키는 임무를 맡았으니 무엇이든 나에게 물어보라."),
    HumanMessage(content="용을 만나려면 어떻게해야하지?"),
]

chat_llm.invoke(messages).content

In [ ]:
# 4-1. Embedding 모델 연결

from langchain_openai import AzureOpenAIEmbeddings

embedding_model = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
)

embedding_model.embed_query("안녕하세요")

In [ ]:
# 4-2. 유사도

from numpy import dot
from numpy.linalg import norm

def cos_sim(word1, word2):
    em1 = embedding_model.embed_query(word1)
    em2 = embedding_model.embed_query(word2)
    return dot(em1, em2)/(norm(em1)*norm(em2))

In [ ]:
# 5. Chat Prompt Template(Chat Model Prompt)

from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

system_template = """
한국어 문장을 {output_language}로 번역하는 인공지능.
번역된 문장의 한글 발음을 괄호 안에 함께 제공한다.
예) Hi(하이), こんにちは(곤니치와)
"""

system_message_prompt_template = SystemMessagePromptTemplate.from_template(
    system_template)

human_template = "{input_text}"

human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt_template, human_message_prompt_template])

final_prompt = chat_prompt_template.format_prompt(output_language="영어",
                          input_text="오늘 날씨 정말 좋네요.").to_messages()

print(final_prompt)

In [ ]:
chat_llm.invoke(final_prompt).content

In [ ]:
# 6. PromptTemplate ⭐️⭐️⭐️
from langchain import PromptTemplate

template = """
한국어 문장 {input_text}을 {output_language}로 번역하는 인공지능.
번역된 문장의 한글 발음을 괄호 안에 함께 제공한다.
예) Hi(하이), こんにちは(곤니치와)
"""

translate_template = PromptTemplate(
    input_variables=["input_text", "output_language"],
    template=template
)

final_prompt = translate_template.format_prompt(input_text="오늘 날씨가 정말 좋아요", 
                                                output_language="일본어").to_messages()
print(final_prompt)

In [ ]:
chat_llm.invoke(final_prompt).content

In [ ]:
# 6-2. Few shot learning

from langchain.prompts import FewShotPromptTemplate
# fewshot 예시 문장
examples = [
    {"input": "영어 : 이 와인 정말 맛있네요", "output": "This wine is really delicious.(발음:디스 와인 이즈 리얼리 딜리셔스)"},
    {"input": "프랑스어 : 이 와인 정말 맛있네요", "output": "Ce vin est vraiment délicieux.(발음:세 반 에트랑제 드리슈)"},
    {"input": "독일어 : 이 와인 정말 맛있네요 ", "output": "Dieser Wein schmeckt wirklich lecker.(발음:디저 바인 샴케 비역클러)"},
    {"input": "일본어 : 이 와인 정말 맛있네요", "output": "このワインは本当においしいですね。(발음:코노 와인와 혼토니 오이시이 데스 네)"},
    {"input": "이태리어 : 이 와인 정말 맛있네요", "output": "Questo vino è davvero delizioso.(발음:쿼스토 비노 에 디버로 델리조오소)"},
    {"input": "이태리어 : 화장실이 어디인가요?", "output": "Dov'è il bagno?(발음:도베 이르 바뇨?"},
    {"input": "일본어 : 화장실이 어디인가요?", "output": "トイレはどこですか？(발음:토이레와 도코데스카?)"},
    {"input": "독일어 : 화장실이 어디인가요?", "output": "Entschuldigung, wo ist die Toilette?(발음:앤쇨룰디귄, 보이스 디 토일레트?)"},
    {"input": "프랑스어 : 화장실이 어디인가요?", "output": "Où sont les toilettes?(발음:우 솽 레 투알레트?)"},
    {"input": "영어 : 화장실이 어디인가요?", "output": "Where is the restroom?(발음:웨어 이즈 더 레스트룸?)"},
]

# fewshot 예시 문장+템플릿 결합
example_prompt = PromptTemplate(input_variables=["input", "output"],
                                template="문장을 주어진 언어로 번역한다. 번역된 문장의 한국어 발음을 괄호 안에 함께 표기한다. : {input} -> {output}")

fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="문장을 주어진 언어로 번역하고 한글 발음을 괄호안에 표기한다.  : {input} -> ",
    input_variables=["input"]
)

final_prompt=fewshot_prompt.format_prompt(input="일본어 : 날씨가 너무 더워요").to_messages()
print(final_prompt)

In [ ]:
chat_llm.invoke(final_prompt).content

In [ ]:
# 7-1. Chain-PromptTemplate 연결
# LangChain without LCEL:

from langchain.chains import LLMChain 
from langchain.schema import StrOutputParser

output_parser = StrOutputParser()

# 체인 = 모델 + 프롬프트 + Parser
chain = LLMChain(llm=chat_llm, 
                 prompt=fewshot_prompt,
                 output_parser=output_parser)

#### LangChain Expression Language (LCEL)
* [LCEL 공식 문서 바로가기](https://python.langchain.com/v0.1/docs/expression_language/)
* [LCEL Parsers 공식 문서](https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/)

In [ ]:
# 7-2. LCEL(LangChain Express Language) 활용 코드
# (model(prompt)) -> output_parser(model(prompt))

chain = fewshot_prompt | chat_llm | output_parser
com_chain = fewshot_prompt | com_llm | output_parser

In [ ]:
# 7-3. Batch : 여러개 인풋 활용

input_list = [
    {"input" : "영어 : 이 와인 정말 맛있네요"},
    {"input" : "이태리어 : 이 와인 정말 맛있네요"},
    {"input" : "프랑스어 : 이 와인 정말 맛있네요"}
]

#### Simple Sequential Chain

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*Qrct9oGPZklQSWwlPNKfqQ.png)
[Image Reference](https://faun.pub/langchain-in-chains-11-chains-33b9f3c2d217)

In [ ]:
# 8-1. 두 개 체인 연결 
# without LCEL

# SimpleSequentialChain: 하나의 input variable 만 사용가능
from langchain.chains import SimpleSequentialChain

# 첫번째 체인(한국어->외국어)
fisrt_chain = LLMChain(llm = chat_llm,
                 prompt = fewshot_prompt)

# 두번째 체인
second_prompt = PromptTemplate(
    input_variables=["foreign_language"],
    template="번역된 문장과 동일한 뜻의 유사 표현 5개와 번역된 문장의 한국어 발음 표기를 괄호 안에 표기한다: {foreign_language}",
)

second_chain = LLMChain(llm=chat_llm, prompt=second_prompt)

# 두가지 체인 결합
overall_chain = SimpleSequentialChain(chains=[fisrt_chain, second_chain], verbose=True)

In [ ]:
# 8-2. With LCEL

chain1 = fewshot_prompt | chat_llm 
chain2 = second_prompt | chat_llm 

overall_chain = chain1 | chain2 | output_parser

In [ ]:
# 줄바꿈 custom parser
def line_parse(output) -> str:
    lines = output.split('\n')
    return lines

#### SequentialChain
![]()

In [ ]:
# 9-1. 여러개 인풋 사용 가능한 Sequential Chain 
# without LCEL

from langchain.chains import SequentialChain

# 첫번째 체인
menu_template = """
신메뉴 이름 {name}이 제공됩니다.
주재료는 {ingredients}입니다.
주어진 정보를 바탕으로 배달 서비스에 쓰일 신메뉴 소개글을 100자로 작성하시오
"""

menu_prompt = PromptTemplate(
    input_variables=['name', 'ingredients'],
    template=menu_template
)

menu_chain = LLMChain(llm=com_llm,
                 prompt=menu_prompt,
                 output_key="description")

# 두번째 체인
review_template = """
다음 신메뉴를 배달 주문으로 먹은 손님의 긍정 한줄 리뷰 3개 작성 
{description}
"""

review_prompt = PromptTemplate(
    input_variables=["description"],
    template=review_template
)

review_chain = LLMChain(llm=chat_llm,
                     prompt=review_prompt,
                     output_key="review")

overall_chain = SequentialChain(
    chains=[menu_chain, review_chain],
    input_variables=["name", "ingredients"],
    output_variables=["description", "review"],
    verbose=True)

In [ ]:
overall_chain.invoke({"name":"", "ingredients": ""})

In [ ]:
# 9-2. SequentialChain with LCEL

# 첫번째 체인
menu_template = """
신메뉴 이름 {name}이 제공됩니다.
주재료는 {ingredients}입니다.
주어진 정보를 바탕으로 배달 서비스에 쓰일 신메뉴 소개글을 100자로 작성하시오
"""

menu_prompt = PromptTemplate(
    input_variables=['name', 'ingredients'],
    template=menu_template
)


# 두번째 체인
review_template = """
다음 신메뉴를 배달 주문으로 먹은 손님의 긍정 한줄 리뷰 3개 작성 
{description}
"""

review_prompt = PromptTemplate(
    input_variables=["description"],
    template=review_template
)

chain1 = menu_prompt | chat_llm
chain2 = review_prompt | chat_llm 

overall_chain = {"description" : chain1 } | chain2

In [ ]:
# 중간 소개글 출력 함수
def run_chain_with_print(output) -> dict:
    print("소개글:", output.content)
    print("리뷰:")
    return {"description" : output}

overall_chain = chain1 | run_chain_with_print | chain2 | output_parser

#### LangChain Memory
![](https://python.langchain.com/v0.1/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

* [LangChain Memory 공식 문서](https://python.langchain.com/v0.1/docs/modules/memory/)

In [ ]:
# 10-1. LangChain Memory

from langchain.memory import ConversationBufferMemory

# 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history")

# 메모리 값 저장1
memory.chat_memory.add_user_message("안녕, 내 이름은 xx야!")
memory.chat_memory.add_ai_message("안녕, 내 이름은 yy야")

# 메모리 값 저장2
memory.save_context(
    {"input" : "안녕, 내 이름은 aa야"},
    {"output" : "반가워, 내이름은 bb야"}
)

# 저장된 메모리 출력
memory.load_memory_variables({})

In [ ]:
# 10-2. Chain-Memory 연결 without LCEL

memory_template = """사용자의 질문에 유용한 정보를 알려주는 챗봇 도우미

이전 대화 기록 : {chat_history}
사용자 질문 : {question}
답변 : 
"""

# prompte template 생성
memory_prompt = PromptTemplate(
    input_variables=["chat_history", "question"],
    template=memory_template
)
# memory_prompt = PromptTemplate.from_template(memory_template)

# 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history")
# Chain 생성
memory_chain = LLMChain(llm=com_llm,
                        prompt=memory_prompt,
                        verbose=True,
                        memory=memory)

In [ ]:
# 10-3. Chain-Memory 연결2 without LCEL : ChatPrompt

from langchain_core.prompts import (
    MessagesPlaceholder,
)

memory_prompt = ChatPromptTemplate(
    messages=[
        SystemMessage(content="RPG 게임 캐릭터 재키, 성을 지키는 용감한 용사로 유저의 질문에 반말 구어체로 짧게 답한다"),
        HumanMessage(content="안녕?"),
        AIMessage(content="용감한 자여, 전설의 성에 온걸 환영한다!"),
        HumanMessage(content="너의 이름은 뭐니?"),
        AIMessage(content="사람들은 나를 재키라 부르지. 성을 지키는 임무를 맡았으니 무엇이든 나에게 물어보라."),

        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)


memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chat_chain = LLMChain(llm=chat_llm,
                      prompt=memory_prompt,
                      verbose=True,
                      memory=memory
)

In [ ]:
# 10-4. Memory Chain with LCEL
from langchain_core.runnables import RunnablePassthrough

# 메모리 초기화
memory.clear()

# 메모리 불러오기 함수(사용하지 않는 인자 _ 생성)
def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


# RunnablePassthrough.assign : chat_history라는 변수에 함수 실행 결과값 할당
lcel_chain = RunnablePassthrough.assign(chat_history=load_memory) | memory_prompt | chat_llm | output_parser

def invoke_lcel_chain(question):
    result = lcel_chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result},
    )
    print(result)


In [ ]:
# 10-5. lcel 질문
invoke_lcel_chain("")

In [ ]:
# 대화 기록 
load_memory(_)